<img align='left' src = '../images/linea.png' width=150 style='padding: 20px'> 

# Tutorial: particionamento de dados no formato HATS e cross-matching com a biblioteca LSDB

Passo-a-passo para conversão de catálogos astronômicos para o formato HATS e execução de cross-matching utilizando a biblioteca LSDB. 

Contatos: Luigi Silva ([luigi.silva@linea.org.br](mailto:luigi.silva@linea.org.br)); Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br)).

Última verificação: 27/08/2024




#### Reconhecimentos

'_Este notebook utilizou recursos computacionais da Associação Laboratório Interinstitucional de e-Astronomia (LIneA) com o apoio financeiro do INCT do e-Universo (Processo n.º 465376/2014-2)._' 

'_Este notebook se baseia nas bibliotecas e documentações do projeto LSST Interdisciplinary Network for Collaboration and Computing (LINCC) Frameworks, principalmente as bibliotecas hats, hats_import e lsdb. O projeto LINCC Frameworks é apoiado pelo Schmidt Sciences. Ele também é baseado em trabalhos apoiados pela National Science Foundation sob o Subsídio nº AST-2003196. Além disso, ele recebe apoio do DIRAC Institute do Departamento de Astronomia da Universidade de Washington. O DIRAC Institute é apoiado por meio de doações do Charles and Lisa Simonyi Fund for Arts and Sciences, e pelo Washington Research Foundation._'

# Introdução

## Links de referência das bibliotecas de interesse principal

Muitos dos textos contidos neste notebook foram extraídos, ou baseados, nos textos das documentações e repositórios das bibliotecas de interesse principal (hats, hats_import e lsdb). A seguir, temos os links para os repositórios e documentações destas bibliotecas. 

### Repositórios 
```lsdb```: https://github.com/astronomy-commons/lsdb <br>
```hats_import```: https://github.com/astronomy-commons/hats-import <br>
```hats```: https://github.com/astronomy-commons/hats

### Documentações
```lsdb```: https://lsdb.readthedocs.io/en/stable/ <br>
```hats_import```: https://hats-import.readthedocs.io/en/stable/ <br>
```hats```: https://hats.readthedocs.io/en/stable/ 

## HEALPix

As bibliotecas hats, hats_import e lsdb utilizam o conceito do HEALPix.

"HEALPix é um acrônimo para Hierarchical Equal Area isoLatitude Pixelization de uma esfera. Como sugerido no nome, essa pixelização produz uma subdivisão de uma superfície esférica na qual cada pixel cobre a mesma área de superfície que todos os outros pixels. A figura abaixo mostra a partição de uma esfera em resoluções progressivamente mais altas, da esquerda para a direita. A esfera verde representa a menor resolução possível com a partição base do HEALPix da superfície esférica em 12 pixels de tamanho igual. A esfera amarela tem uma grade HEALPix de 48 pixels, a esfera vermelha tem 192 pixels e a esfera azul tem uma grade de 768 pixels (resolução de ~7,3 graus).

<center> <img src="https://healpix.jpl.nasa.gov/images/healpixGridRefinement.jpg" width="600"> </center>

Outra propriedade da grade HEALPix é que os centros dos pixels, representados pelos pontos pretos, ocorrem em um número discreto de anéis de latitude constante. O número de anéis de latitude constante depende da resolução da grade HEALPix. Para as esferas verde, amarela, vermelha e azul mostradas, há 3, 7, 15 e 31 anéis de latitude constante, respectivamente." ([HEALPix - NASA](https://healpix.jpl.nasa.gov/))

## HATS

"Um catálogo [HATS (Hierarchical Adaptive Tiling Scheme)](https://github.com/astronomy-commons/hats) é uma partição de objetos em uma esfera. Seu propósito é armazenar dados de grandes levantamentos astronômicos, mas provavelmente poderia ser usado para outros casos de uso onde se tenha grandes volumes de dados com algumas propriedades esféricas." ([HATS - GitHub](https://github.com/astronomy-commons/hats))

#### Esquema de Particionamento
"Nos catálogos no formato HATS, é utilizado o HEALPix (Hierarchical Equal Area isoLatitude Pixelization) para a pixelização esférica e as partições são dimensionadas de forma adaptativa com base no número de objetos.

Em áreas do céu com mais objetos, são usados pixels menores, de modo que todos os pixels resultantes contenham contagens similares de objetos (dentro de uma ordem de magnitude)." ([HATS - Docs](https://hats.readthedocs.io/en/stable/guide/directory_scheme.html))

#### Estrutura de Arquivos
"O leitor do catálogo espera encontrar arquivos de acordo com a seguinte estrutura particionada: " ([HATS - Docs](https://hats.readthedocs.io/en/stable/guide/directory_scheme.html))

```
_ /path/to/catalogs/<catalog_name>/
   |__ partition_info.csv
   |__ properties
   |__ dataset/
       |__ _common_metadata
       |__ _metadata
       |__ Norder=1/
       |   |__ Dir=0/
       |       |__ Npix=0.parquet
       |       |__ Npix=1.parquet
       |__ Norder=J/
           |__ Dir=10000/
               |__ Npix=K.parquet
               |__ Npix=M.parquet
```

## Cross-matching espacial

O cruzamento espacial, ou *cross-matching* espacial, entre diferentes catálogos astronômicos consiste em identificar e comparar objetos astronômicos de uma mesma região do céu, porém provenientes de diferentes observações.

O *cross-matching espacial* entre diferentes catalógos é muito útil. Um exemplo desta utilidade pode ser encontrado na elaboração de conjuntos de treinamento para algoritmos de *machine learning* que calculam *redshifts* fotométricos. Esses conjuntos de treinamento podem ser elaborados a partir do cruzamento de objetos de um catálogo fotométrico (de onde serão extraídos os *features*) com objetos de um catálogo espectroscópico (de onde serão extraídos os nossos *true redshifts*).

### Escalabilidade

Com o grande volume de dados dos levantamentos atuais e futuros, como o [Rubin Observatory Legacy Survey of Space and Time (LSST)](https://rubinobservatory.org/explore/how-rubin-works/lsst), o armazenamento e manipulação destes dados é um grande desafio. Catálogos com bilhões de objetos e tamanho de vários terabytes são desafiadores de armazenar e manipular porque exigem hardware de última geração. Processá-los é caro, tanto em termos de tempo de execução quanto de consumo de memória, e realizá-lo em uma única máquina tornou-se impraticável. ([LSDB - Docs](https://docs.lsdb.io/en/stable/tutorials/filtering_large_catalogs.html))

A biblioteca [LSDB (Large Survey DataBase)](https://github.com/astronomy-commons/lsdb) é uma solução que permite a execução escalável de algoritmos. Ele lida com carregamento, consulta, filtragem e cruzamento de dados astronômicos (no formato HATS) em um ambiente distribuído. O *framework* que permite essa escalabilidade, utilizado pelo LSDB, é o Dask, que aproveita as capacidades de computação distribuída. Com Dask, as operações definidas em um fluxo de trabalho são adicionadas a um gráfico de tarefas que otimiza sua ordem de execução. As operações não são imediatamente computadas - somos nós que decidimos quando iremos computá-las. Assim que iniciamos as computações, o Dask distribui a carga de trabalho entre seus vários *dask workers*, distribuídos entre os nós de um Cluster, por exemplo, e as tarefas são executadas de maneira eficiente em paralelo. ([LSDB - Docs](https://docs.lsdb.io/en/stable/tutorials/filtering_large_catalogs.html))

### Objetos nas bordas

A biblioteca LSDB usa os catálogos no formato HATS como forma de organizar os dados espacialmente para, dentre outras coisas, conseguir carregar todos os pontos vizinhos de maneira simultânea, o que é essencial para comparações precisas. No entanto, há uma limitação: nas bordas de cada pixel, alguns pontos serão perdidos. Isso significa que, para operações que exigem comparações com pontos vizinhos, como o cruzamento de dados, o processo pode perder algumas correspondências para pontos próximos às bordas das partições, porque nem todos os pontos próximos são incluídos ao analisar uma partição de cada vez. ([LSDB Docs](https://lsdb.readthedocs.io/en/stable/tutorials/margins.html))

<center> <img src="https://lsdb.readthedocs.io/en/stable/_images/pixel-boundary-example.png" width="600"> </center>
<center><legend>Aqui vemos um exemplo de uma fronteira entre pixels HEALPix, onde os pontos verdes estão em uma partição e os pontos vermelhos em outra. Trabalhando com uma partição de cada vez, perderíamos correspondências potenciais com pontos próximos à fronteira. (<a href="https://lsdb.readthedocs.io/en/stable/tutorials/margins.html">LSDB - Docs</a>) </legend></center> <br>

Para resolver isso, poderíamos tentar carregar também as partições vizinhas para cada partição que cruzarmos. No entanto, isso significaria carregar muitos dados desnecessários, o que desaceleraria as operações e causaria problemas de falta de memória. Então, para cada catálogo, também criamos um cache de margem. Isso significa que, para cada partição, criamos um arquivo que contém os pontos no catálogo dentro de uma certa distância da borda do pixel. ([LSDB Docs](https://lsdb.readthedocs.io/en/stable/tutorials/margins.html))

<center> <img src="https://lsdb.readthedocs.io/en/stable/_images/margin-pix.png" width="600"> </center>
<center><legend>Um exemplo de um cache de margem (laranja) para o mesmo pixel anterior, em verde. O cache de margem para este pixel contém os pontos dentro de uma distância de 10 arcsec da fronteira. (<a href="https://lsdb.readthedocs.io/en/stable/tutorials/margins.html">LSDB - Docs</a>) </legend></center> <br>

# Importação das bibliotecas e configurações

Antes da instalação das bibiliotecas necessárias para esse notebook, é **recomendado criar um ambiente virtual**. Para isso, você pode seguir os passos contidos na documentação da biblioteca [LSDB](https://docs.lsdb.io/en/stable/getting-started.html) ou os passos do notebook de tutorial ```3-conda-env.ipynb``` contido no [repositório de tutorial do LIneA](https://github.com/linea-it/jupyterhub-tutorial). Após isso, para utilizar esse ambiente virtual como um kernel no Jupyter Notebook, são necessários os passos a seguir:

<p style="background-color:black; color:white;">
    <font face="Courier New">
        conda install -c anaconda ipykernel 
    </font>
</p>

<p style="background-color:black; color:white;">
    <font face="Courier New">
        python -m ipykernel install --user --name=NOME-DO-SEU-AMBIENTE-VIRTUAL
    </font>
</p>

Estes comandos vão fazer com que o ambiente criado seja disponibilizado como um kernel para o Jupyter Notebook.

As instruções de instalação para as bibliotecas de interesse principal podem ser encontradas nas documentações do [LSDB](https://docs.lsdb.io/en/stable/getting-started.html) e do [HATS Import](https://hats-import.readthedocs.io/en/stable/).


Requisitos para este notebook:

* **Bibliotecas gerais**: os, sys, math, numpy, time, pathlib.
* **Bibliotecas astronômicas:** astropy.
* **Bibliotecas de computação paralela**: dask.
* **Bibliotecas de visualização**: bokeh, holoviews, geoviews, datashader, matplotlib.
* **Bibliotecas de interesse principal:** hats, hats_import, lsdb.
* **Bibliotecas de manipulação de dados**: pandas.
* **Bibliotecas para a obtenção de dados**: dblinea.

* **Arquivo auxiliar**: [des-round19-poly.txt](https://github.com/kadrlica/skymap/blob/master/skymap/data/des-round19-poly.txt) (contorno da área coberta pelo levantamento do DES DR2, i.e., DES _footprint_, 2019 version).

Precisamos fazer o download do arquivo `des-round19-poly.txt` do repositório [kadrlica/skymap](https://github.com/kadrlica/skymap) no GitHub.

In [ ]:
! wget https://raw.githubusercontent.com/kadrlica/skymap/master/skymap/data/des-round19-poly.txt -O des-round19-poly.txt 

## Importações

Vamos importar as bibliotecas necessárias.

In [ ]:
###################### GENERAL ######################
import os
import sys
import math
import numpy as np
import time
from pathlib import Path

In [ ]:
###################### ASTRONOMY ######################
### ASTROPY
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
###################### PARALLEL COMPUTING ######################
### DASK
from dask.distributed import Client

In [ ]:
###################### VISUALIZATION ######################
### BOKEH
import bokeh
from bokeh.io import output_notebook, show
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.palettes import Viridis256

### HOLOVIEWS
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import rasterize, dynspread

### GEOVIEWS
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

### MATPLOTLIB
import matplotlib.pyplot as plt

In [ ]:
###################### HATS AND LSDB ######################
### HATS
## Explore the HATS catalogs and plot sky maps
import hats
from hats.catalog import Catalog
from hats.inspection import plot_pixels

## For converting the data to HATS format and generate margin caches
import hats_import
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client  

### LDSB
import lsdb

In [ ]:
###################### DATA MANAGEMENT ######################
### PANDAS
import pandas as pd

In [ ]:
###################### DATA ACCESS ######################
### DB LIneA
from dblinea import DBBase

A seguir, são impressas as versões do Python, Numpy, Bokeh e Holoviews:

In [ ]:
print('Python version: ' + sys.version)
print('Numpy version: ' + np.__version__)
print('Bokeh version: ' + bokeh.__version__)
print('HoloViews version: ' + hv.__version__)
print('hats-import version: ' + hats_import.__version__)
print('lsdb version: ' + lsdb.__version__)

## Configurações

Definindo o cliente Dask.

In [ ]:
client = Client()

Definindo o número de linhas que o pandas irá exibir.

In [ ]:
pd.set_option('display.max_rows', 10)

Configurando o holoviews e o geoviews para trabalhar com o bokeh:

In [ ]:
hv.extension('bokeh')

In [ ]:
gv.extension('bokeh')

Configurando os plots do bokeh para serem em linha:

In [ ]:
output_notebook()

Configurando os plots do matplotlib para serem em linha:

In [ ]:
%matplotlib inline

## Leitura dos dados do footprint do DES

A seguir, vamos ler o footprint do DES DR2 do arquivo `des-round19-poly.txt` e imprimir os mínimos e máximos de R.A. e DEC.

In [ ]:
foot_ra, foot_dec = np.loadtxt('des-round19-poly.txt', unpack=True)

print("R.A. AND DEC COORDINATES, BEFORE USING SKYCOORD")
print(f"R.A. min: {foot_ra.min():.2f} | R.A. max: {foot_ra.max():.2f}")
print(f"DEC min: {foot_dec.min():.2f} | DEC max: {foot_dec.max():.2f}")

Depois de ler o footprint, definimos a classe SkyCoord da biblioteca Astropy usando as coordenadas R.A. e DEC do footprint. Com o SkyCoord, temos uma interface flexível para representação, manipulação e transformação de coordenadas celestes entre sistemas. Usamos também o módulo de unidades do Astropy; em ```u.degree```, por exemplo, indicamos que as coordenadas estão em graus. Além disso, usamos o método ```wrap_at``` para garantir que as coordenadas estejam no intervalo $[-180,180)$.

In [ ]:
foot_coords = SkyCoord(ra=foot_ra*u.degree, dec=foot_dec*u.degree, frame='icrs')
foot_df = pd.DataFrame({'foot_ra': np.array(foot_coords.ra.wrap_at(180*u.degree)), 
                        'foot_dec': np.array(foot_coords.dec)})

print("R.A. AND DEC COORDINATES, AFTER USING SKYCOORD")
print(f"R.A. min: {foot_df['foot_ra'].min():.2f} | R.A. max: {foot_df['foot_ra'].max():.2f}")
print(f"DEC min: {foot_df['foot_dec'].min():.2f} | DEC max: {foot_df['foot_dec'].max():.2f}")

## Função para gerar os gráficos de distribuição espacial

A seguir, temos uma função que retorna os gráficos de distribuição espacial, dadas as coordenadas R.A. e DEC dos objetos. Essa função será usada posteriormente nos plots.

In [ ]:
def plot_spatial_distribution(ra_coords, dec_coords, title, 
                              longitudes_ticks=np.arange(30, 360, 30), 
                              latitudes_ticks=np.arange(-75, 76, 15),
                              height=500, width=1000, padding=0.05,
                              xlabel='R.A.', ylabel='DEC', 
                              show_des_footprint=True, 
                              show_grid=True, 
                              show_labels=True):
    '''Função para exibir os gráficos de distribuição espacial de objetos.
        
        Argumentos:
        ra_coords (pandas.core.series.Series, numpy.ndarray ou list): coordenadas R.A. do objeto, em graus.
        dec_coords (pandas.core.series.Series, numpy.ndarray ou list): coordenadas DEC do objeto, em graus.
        title (str): título a ser exibido no gráfico.
        longitudes_ticks (numpy.ndarray): ticks de longitude a serem exibidos no gráfico. PADRÃO: np.arange(30, 360, 30)
        latitudes_ticks (numpy.ndarray): ticks de latitude a serem exibidos no gráfico. PADRÃO: np.arange(-75, 76, 15)
        height (float): parâmetro relacionado às dimensões do plot. PADRÃO: 500
        width (float): parâmetro relacionado às dimensões do plot. PADRÃO: 1000
        padding (float): parâmetro relacionado ao posicionamento do plot. PADRÃO: 0.05
        xlabel (str): nome do eixo-x. PADRÃO: 'R.A.'
        ylabel (str): nome do eixo-y. PADRÃO: 'DEC'
        show_des_footprint (boolean): se mostra ou não o footprint do DES. PADRÃO: True
        show_grid (boolean): se mostra ou não as linhas de grid. PADRÃO: True
        show_labels (boolean): se mostra ou não os ticks de latitude e longitude. PADRÃO: True
    '''
    
    ### Definindo os ticks de R.A. e DEC. para os plots.
    longitudes = longitudes_ticks
    latitudes = latitudes_ticks

    lon_labels = [f"{lon}°" for lon in longitudes]
    lat_labels = [f"{lat}°" for lat in latitudes]

    labels_data = {
        "lon": list(np.flip(longitudes)) + [-180] * len(latitudes),
        "lat": [0] * len(longitudes) + list(latitudes),
        "label": lon_labels + lat_labels,
    }

    df_labels = pd.DataFrame(labels_data)

    labels_plot = gv.Labels(df_labels, kdims=["lon", "lat"], vdims=["label"]).opts(
        text_font_size="12pt",
        text_color="black",
        text_align='right',
        text_baseline='bottom',
        projection=crs.Mollweide()
    )

    ### Definindo as linhas de grid.
    grid = gf.grid()

    ### Definindo a curva do footprint do DES.
    ### Aqui, multiplicamos as coordenadas 'ra' por (-1) para que o plot fique invertido em R.A. Essa é uma convenção muito utilizada.
    ra_dec_foot = gv.Path(((-1)*foot_df['foot_ra'], foot_df['foot_dec'])).opts(line_width=3, color='orange')

    ### Definindo a classe SkyCoord do astropy para manipulação das coordenadas astronômicas.
    ra_dec_coords = SkyCoord(ra=np.array(ra_coords)*u.degree, dec=np.array(dec_coords)*u.degree, frame='icrs')

    ### Usando o método wrap_at para garantir que as coordenadas estejam no intervalo [-180, 180).
    ra_dec_coords = pd.DataFrame({'ra_coords': np.array(ra_dec_coords.ra.wrap_at(180*u.degree)), 
                                  'dec_coords': np.array(ra_dec_coords.dec)})

    ### Definindo o objeto points do geoviews.
    ### Novamente, multiplicamos as coordenadas 'ra' por (-1) para que o plot fique invertido em R.A. Essa é uma convenção muito utilizada.
    ra_dec_points = gv.Points(((-1)*ra_dec_coords['ra_coords'], ra_dec_coords['dec_coords']), kdims=['ra', 'dec'])

    ### Aplicando a projeção Mollweide.
    projected = gv.operation.project(ra_dec_points, projection=crs.Mollweide())

    ### Aplicando o datashader sobre os pontos.
    dsh_points = dynspread(rasterize(projected).opts(cmap="Viridis", cnorm='log'))
    dsh_points = dsh_points.opts(width=width, height=height, padding=padding, title=title, toolbar='above', colorbar=True, tools=['box_select'])

    ### Exibindo o plot.
    if show_des_footprint==True:
        return(dsh_points * ra_dec_foot * grid * labels_plot)
    elif show_grid==True:
        return(dsh_points * grid * labels_plot)
    elif show_labels==True:
        return(dsh_points * labels_plot)
    else:
        return(dsh_points)

# Caracterização da amostra espectroscópica de exemplo

Para executar o cross-matching posteriormente, **usaremos como dados espectroscópicos uma amostra de objetos do 2dFLenS.**

|Nome do levantamento <br> (link para o website)| Número de **redshifts** na <br>amostra original | Referência <br> (link para o artigo) |
|---|:-:|---|
|[2dFLenS](http://2dflens.swin.edu.au/) |70,079| [Blake et al. 2016](https://ui.adsabs.harvard.edu/abs/2016MNRAS.462.4240B/abstract)|

A amostra que será utilizada será extraída de uma tabela chamada **public_specz_compilation** via biblioteca DBLIneA. Essa tabela contém um compilado de catálogos de diferentes levantamentos, os quais foram coletados ao longo dos anos de operação do Dark Energy Survey (DES) e agrupados sistematicamente pela ferramenta DES Science Portal (pipeline Spectroscopic Sample) para formar a base de um conjunto de treinamento para algoritmos de cálculo de *redshifts* fotométricos baseados em machine learning. Temos, no total, dados de 28 levantamentos, como 2DF, 2dFLenS, 3DHST, 6DF, etc.

A seguir, podemos ver as 5 primeiras linhas desse compilado de *redshifts* fotométricos, a estatística básica dos dados e um plot simples de sua distribuição espacial.

In [ ]:
db = DBBase()
schema = 'des_dr2'  
tablename = 'public_specz_compilation'

query = f'SELECT * FROM {schema}.{tablename}'
specz_compilation_data = db.fetchall_df(query)

In [ ]:
specz_compilation_data.head()

In [ ]:
specz_compilation_data.describe()

In [ ]:
ra_coords = specz_compilation_data['ra']
dec_coords = specz_compilation_data['dec']
title = 'Distribuição espacial - Todos os objetos contidos na tabela Public spec-z compilation'

plot_spatial_distribution(ra_coords, dec_coords, title)

**Observação: pode acontecer um bug de reenderização nos plots que subestime/superestime os objetos por pixel. Um simples clique na ferramenta "reset", na barra de ferramentas do Bokeh na parte superior do plot, deve resolver o problema. Se não resolver, certifique-se de que todos os pacotes e extensões foram instaladas corretamente, de forma que os gráficos estejam dinâmicos (reenderizam conforme o zoom dado). Em caso de dúvida, verifique o arquivo de instruções ```instructions_hispcat_lsdb_tutorial.md```, que está no mesmo diretório deste notebook.**

Além de combinar todos os catálogos em uma única tabela, o _pipeline_ Spectroscopic Sample também homogeneiza as várias flags de qualidade originais dos catálogos em um único sistema (`flag_des`) baseado nos parâmetros usados no levantamento OzDES ([Yuan et al., 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.452.3047Y/abstract)). Em resumo, as flags significam:

|flag_des| Significado |
|--- |---|
|1 | redshift desconhecido |
|2 | palpite não confiável |
|3 | 95% de confiança |
|4 | 99% de confiança |

Como a tabela **public_specz_compilation** contém um compilado de *redshifts* espectroscópicos de vários levantamentos astronômicos, muitos desses levantamentos observaram regiões comuns do céu. Ao agrupar todas as medidas em um catálogo único, geralmente há medidas múltiplas de *redshift* espectroscópico para um mesmo objeto. Para identificar esses casos, o *pipeline* Spectroscopic Sample fez uma combinação espacial entre as coordenadas equatoriais de "todos contra todos" com um raio de busca de 1.0 *arcsec* de cada objeto. Então, ele aplicou uma seleção para manter apenas uma medida para cada objeto extragaláctico presente na amostra, seguindo o critério abaixo para escolha e desempate:

1. medida com a maior _flag_ de qualidade (`flag_des`)
2. medida com o menor erro no *redshift* (`err_z`)
3. medida obtida pelo levantamento mais recente

Um corte de qualidade também foi aplicado onde apenas objetos com flag_des ⩾ 3 foram incluídos no compilado.


Portanto, como a amostra do 2dFLenS que usaremos será um **subconjunto dos dados** contidos na tabela **public spec-z compilation**, estamos pegando, nessa amostra, **apenas aqueles objetos que permaneceram após a seleção considerando todos os outros catálogos presentes no compilado de *redshifts* fotométricos, e apenas os objetos com flag_des ⩾ 3.**

## Todos os objetos do 2dFLenS, contidos no produto Public spec-z compilation

Primeiramente, vamos filtrar os dados do compilado para obter apenas aqueles referentes ao 2dFLenS.

In [ ]:
df_specz_sample = specz_compilation_data[specz_compilation_data['survey']=='2dFLenS']

A seguir, temos as primeiras 5 linhas da tabela contendo a amostra.

In [ ]:
df_specz_sample.head()

Temos também as estatísticas básicas dos dados.

In [ ]:
df_specz_sample.describe()

Em seguida, fazemos o plot de distribuição de objetos.

In [ ]:
ra_coords = df_specz_sample['ra']
dec_coords = df_specz_sample['dec']
title = 'Distribuição espacial - Todos os objetos do 2dFLenS, contidos na tabela Public spec-z compilation'

plot_spatial_distribution(ra_coords, dec_coords, title)

**Observação: pode acontecer um bug de reenderização nos plots que subestime/superestime os objetos por pixel. Um simples clique na ferramenta "reset", na barra de ferramentas do Bokeh na parte superior do plot, deve resolver o problema. Se não resolver, certifique-se de que todos os pacotes e extensões foram instaladas corretamente, de forma que os gráficos estejam dinâmicos (reenderizam conforme o zoom dado). Em caso de dúvida, verifique o arquivo de instruções ```instructions_hispcat_lsdb_tutorial.md```, que está no mesmo diretório deste notebook.**

## Objetos do 2dFLenS filtrados para pertencerem ao footprint do DES

A seguir, faremos um filtro simples para obter os dados do 2dFLenS dentro do footprint do DES. O filtro a seguir irá guardar objetos com $ra<90$ ou $ra>359$, e $dec < -15$.

In [ ]:
df_specz_sample_filtered = df_specz_sample.query("ra > 359 | ra < 90. & dec < -15.")

Mostrando as primeiras 5 linhas da tabela contendo a amostra filtrada.

In [ ]:
df_specz_sample_filtered.head()

Estatística básica desses dados filtrados.

In [ ]:
df_specz_sample_filtered.describe()

Em seguida, fazemos o plot de distribuição de objetos.

In [ ]:
ra_coords = df_specz_sample_filtered['ra']
dec_coords = df_specz_sample_filtered['dec']
title = 'Distribuição espacial - Objetos do 2dFLenS filtrados para pertencerem ao footprint do DES'

plot_spatial_distribution(ra_coords, dec_coords, title)

**Observação: pode acontecer um bug de reenderização nos plots que subestime/superestime os objetos por pixel. Um simples clique na ferramenta "reset", na barra de ferramentas do Bokeh na parte superior do plot, deve resolver o problema. Se não resolver, certifique-se de que todos os pacotes e extensões foram instaladas corretamente, de forma que os gráficos estejam dinâmicos (reenderizam conforme o zoom dado). Em caso de dúvida, verifique o arquivo de instruções ```instructions_hispcat_lsdb_tutorial.md```, que está no mesmo diretório deste notebook.**

Por fim, salvamos os dados em um arquivo. Antes de salvar, no entanto, resetamos os índices para que eles sejam inteiros sequenciais, para que possamos executar o cross-matching posteriormente. Os índices originais da tabela serão salvos na coluna "index".

In [ ]:
! mkdir -p data
! mkdir -p data/input
! mkdir -p data/input/specz

In [ ]:
### Resetando os índices.
df_specz_sample_filtered_reseted = df_specz_sample_filtered.reset_index()

df_specz_sample_filtered_reseted.head()

In [ ]:
### Salvando o arquivo.
df_specz_sample_filtered_reseted.to_parquet('data/input/specz/specz-2dflens-sample.pq', index=False)

# Caracterização da amostra fotométrica de exemplo

Para executar o cross-matching posteriormente, **usaremos como dados fotométricos uma amostra de objetos do DES DR2.**

|Nome do levantamento <br> (link para o website)| Número de objetos na <br>tabela original | Referência <br> (link para o artigo) |
|---|:-:|---|
|[DES DR2](https://des.ncsa.illinois.edu/releases/dr2)|~691 milhões de objetos astronômicos distintos|[DES Collaboration 2021](https://arxiv.org/abs/2101.05765)| 

Para obter esses dados, utilizaremos a biblioteca ```dblinea```. Aqui, nós vamos acessar os dados da tabela **coadd_objects** do catálogo **DES DR2**.

In [ ]:
schema = "des_dr2"  
tablename = "main"

A tabela original tem 215 colunas. O nome e significado de cada coluna pode ser encontrado [aqui](https://des.ncsa.illinois.edu/releases/dr2/dr2-products/dr2-schema). Aqui, iremos utilizar as seguintes colunas:

| Coluna | Significado |
|---|---|
| COADD_OBJECT_ID | Identificador único para os objetos co-adicionados |
| RA | Ascensão reta, com precisão quantizada para indexação (ALPHAWIN_J2000 tem precisão total, mas não indexada) [graus] |
| DEC | Declinação, com precisão quantizada para indexação (DELTAWIN_J2000 tem precisão total, mas não indexada) [graus] |
| MAG_AUTO_{G,R,I,Z,Y}_DERED | Estimativa de magnitude desavermelhada (usando SFD98), para um modelo elíptico baseado no raio de Kron [mag] |
| MAGERR_AUTO_{G,R,I,Z,Y} | Incerteza na estimativa de magnitude, para um modelo elíptico baseado no raio de Kron [mag] |
| FLAGS_{G,R,I,Z,Y} | *Flag* aditiva que descreve conselhos preventivos sobre o processo de extração da fonte. Use menos de 4 para objetos bem comportados |
| EXTENDED_CLASS_COADD | 0: estrelas de alta confiança; 1: estrelas candidatas; 2: principalmente galáxias; 3: galáxias de alta confiança; -9: Sem dados; Usando fotometria Sextractor |

Além disso, a tabela original tem muitos dados. Seria inviável, em termos computacionais, pegar todos os dados que coincidem com a região do 2dFLenS neste notebook. Portanto, iremos restringir a busca para uma região pequena. Aqui, usaremos a região com $7 < R.A < 10$ e $-33 < DEC < -30$.

In [ ]:
### Definindo as coordenadas para a query. Essas variáveis também serão usadas posteriormente nos plots.
ra_min = 7
ra_max = 10
dec_min = -33
dec_max = -30

### Executando a query.
query = (f"SELECT coadd_object_id, ra, dec, mag_auto_g_dered, mag_auto_r_dered, mag_auto_i_dered, mag_auto_z_dered, mag_auto_y_dered, magerr_auto_g, "+
         f"magerr_auto_r, magerr_auto_i, magerr_auto_z, magerr_auto_y, flags_g, flags_r, flags_i, flags_z, flags_y, extended_class_coadd "+ 
         f"FROM {schema}.{tablename} "+
         f"WHERE (dec <= {dec_max} AND dec >= {dec_min} AND ra <= {ra_max} AND ra>={ra_min}) "
        )

In [ ]:
%%time
df_photo_sample = db.fetchall_df(query)

A seguir, temos as primeiras cinco linhas da tabela contendo a amostra.

In [ ]:
df_photo_sample.head()

Temos também as statísticas básicas dos dados.

In [ ]:
df_photo_sample.describe()

A seguir, fazemos um plot da distribuição espacial desses objetos.

In [ ]:
ra_coords = df_photo_sample['ra']
dec_coords = df_photo_sample['dec']
title = 'Distribuição espacial - Amostra do DES DR2'

plot_spatial_distribution(ra_coords, dec_coords, title)

**Observação: pode acontecer um bug de reenderização nos plots que subestime/superestime os objetos por pixel. Um simples clique na ferramenta "reset", na barra de ferramentas do Bokeh na parte superior do plot, deve resolver o problema. Se não resolver, certifique-se de que todos os pacotes e extensões foram instaladas corretamente, de forma que os gráficos estejam dinâmicos (reenderizam conforme o zoom dado). Em caso de dúvida, verifique o arquivo de instruções ```instructions_hispcat_lsdb_tutorial.md```, que está no mesmo diretório deste notebook.**

Por fim, salvamos os dados em um arquivo.

In [ ]:
! mkdir -p data
! mkdir -p data/input
! mkdir -p data/input/photo

In [ ]:
df_photo_sample.to_parquet('data/input/photo/photo-des-dr2-sample.pq', index=False)

# Utilizando o `hats_import`

A biblioteca ```hats_import``` é um utilitário para converter grandes volumes de dados de levantamentos astronômicos para a estrutura HATS. A seguir, iremos utilizá-la para converter os dados espectroscópicos e fotométricos para o formato HATS.

## Instalação do `hats_import` 

A biblioteca hats_import normalmente pode ser instalada via pip com o comando:

```shell
pip install hats-import

```

Para mais detalhes sobre a instalação, veja a documentação [neste link](https://hats-import.readthedocs.io/en/stable/).

## Conversão para o formato HATS dos dados espectroscópicos

Para os passos a seguir, foram utilizados, principalmente, os seguintes exemplos de referência: <br>
https://lsdb.readthedocs.io/en/stable/tutorials/pre_executed/des-gaia.html <br>
https://hats-import.readthedocs.io/en/stable/catalogs/public/sdss.html <br>

Primeiramente, vamos ler o arquivo Parquet que salvamos para conferir a estrutura da tabela.

In [ ]:
pd.read_parquet("data/input/specz/specz-2dflens-sample.pq").head()

Agora, vamos definir os diretórios para salvar os dados no formato HATS.

In [ ]:
! mkdir -p data
! mkdir -p data/hats

In [ ]:
### Diretório dos dados espectroscópicos de input.
_2DFLENS_DIR = Path("data/input/specz/")

### Nomes para os diretórios do catalógo e do cache de margem dos dados espectroscópicos no formato HATS.
_2DFLENS_HATS_NAME = "_2dflens"
_2DFLENS_MARGIN_CACHE_NAME = "_2dflens_margin_cache"

### Definindo os diretórios para os arquivos HATS com base nos nomes definidos anteriormente.
HATS_DIR = Path("data/hats/")
_2DFLENS_HATS_DIR = HATS_DIR / _2DFLENS_HATS_NAME
_2DFLENS_MARGIN_CACHE_DIR = HATS_DIR / _2DFLENS_MARGIN_CACHE_NAME

Vamos, então, converter os dados espectroscópicos para o formato HATS. Para isso, precisamos indicar para o pipeline as colunas contendo os IDs dos objetos, as coordenadas R.A. e DEC, o arquivo de input e o tipo deste arquivo (no nosso caso, um arquivo Parquet) e o nome e o diretório do catálogo HATS de output. 

Além disso, ao criar um novo catálogo utilizando o ```hats_import```, o pipeline tenta criar partições com aproximadamente o memso número de linhas por partição. Esse processo não é perfeito, porém, mesmo assim, o pipeline tenta criar pixels de área menor em áreas mais densas e pixels de área maior em áreas menos densas.

O argumento pixel_threshold é usado para indicar até que ponto o pipeline deve dividir uma certa partição. Ele irá dividir a partição em pixels HEALPix cada vez menores até que o número de linhas fique menor que o pixel_theshold, parando o processo. O processo também pode parar caso hajam tantas divisões que ultrapassem o parâmetro ```highest_healpix_order``` (você pode conferir a ordem máxima padrão [neste link](https://hats-import.readthedocs.io/en/stable/autoapi/hats_import/catalog/arguments/index.html#hats_import.catalog.arguments.ImportArguments.highest_healpix_order)). Se for preciso dividir ainda mais, surgirá um erro na etapa "Binning" e os parâmetros devem ser ajustados.

Para mais detalhes, [veja a documentação](https://hats-import.readthedocs.io/en/stable/catalogs/arguments.html).

Convertendo os dados para o formato hats (**DESCOMENTE A ÚLTIMA LINHA PARA RODAR O PIPELINE**):

In [ ]:
_2dflens_args = ImportArguments(
    sort_columns="index",
    ra_column="ra",
    dec_column="dec",
    input_path=_2DFLENS_DIR,
    file_reader="parquet",
    output_artifact_name=_2DFLENS_HATS_NAME,
    output_path=HATS_DIR,
    pixel_threshold=1_000,
)

### RUN THE PIPELINE (it will fail if the HATS catalog already exists) 
pipeline_with_client(_2dflens_args, client)

Fazendo o plot dos pixels:

In [ ]:
# Read the HATS catalog metadata, it does not load any data, just healpix pixels and other metadata
_2dflens_hats_catalog = hats.read_hats(_2DFLENS_HATS_DIR)
plot_pixels(_2dflens_hats_catalog)

Posteriormente, desejamos fazer o cross-matching desses dados espectrocópicos (2dFLenS) no formato HATS com os dados fotométricos (DES DR2).

O cross-matching, executado pela biblioteca LSDB, não é simétrico, o que significa que a escolha de qual catálogo é o "esquerdo" e qual é o "direito" é crucial. No nosso caso, iremos fazer o cross-matching do DES DR2 (esquerdo) com o 2dFLenS (direito). Essa configuração geralmente permite que múltiplos objetos DES sejam correspondidos a um único objeto do 2dFLenS, um resultado dos caches de margem. Os caches de margem são projetados para evitar a perda de objetos próximos às bordas dos tiles HEALPix. No entanto, eles podem levar a múltiplas correspondências, onde o mesmo objeto do 2dFLenS pode corresponder a um objeto DES em uma partição e a outro objeto DES na partição vizinha que inclui esse objeto do 2dFLenS em seu cache de margem.

Portanto, para o cross-matching, a biblioteca LSDB precisa do cache de margem do catálogo direito para gerar o resultado completo do cruzamento. Sem o cache de margem, os objetos localizados perto das bordas dos tiles Healpix podem ser perdidos no cruzamento. Veja mais detalhes [neste link]( https://lsdb.readthedocs.io/en/stable/tutorials/pre_executed/des-gaia.html).

Gerando o cache de margem para o 2dFLenS (**DESCOMENTE A ÚLTIMA LINHA PARA RODAR O PIPELINE**):

In [ ]:
margin_cache_args = MarginCacheArguments(
    input_catalog_path=_2DFLENS_HATS_DIR,
    output_path=HATS_DIR, 
    margin_threshold=5.0,  # arcsec
    output_artifact_name=_2DFLENS_MARGIN_CACHE_NAME,
)

### RUN THE PIPELINE (it will fail if the margin cache already exists) 
pipeline_with_client(margin_cache_args, client)

## Conversão para o formato hats dos dados fotométricos

Para os passos a seguir, foram utilizados, principalmente, os seguintes exemplos de referência: <br>
https://lsdb.readthedocs.io/en/stable/tutorials/pre_executed/des-gaia.html <br>
https://hats-import.readthedocs.io/en/stable/catalogs/public/sdss.html <br>

Primeiramente, vamos ler o arquivo Parquet que salvamos para conferir a estrutura da tabela.

In [ ]:
pd.read_parquet("data/input/photo/photo-des-dr2-sample.pq").head()

Agora, vamos definir os diretórios para salvar os dados no formato HATS.

In [ ]:
! mkdir -p data
! mkdir -p data/hats

In [ ]:
### Diretório dos dados fotométricos de input.
DES_DIR = Path("data/input/photo/")

### Nome para o diretório do catalógo dos dados fotométricos no formato HATS.
DES_HATS_NAME = "des_dr2"

### Definindo o diretório para os arquivos HATS com base no nome definido anteriormente.
HATS_DIR = Path("data/hats/")
DES_HATS_DIR = HATS_DIR / DES_HATS_NAME

Da mesma forma que foi feito para os dados espectroscópicos, vamos, a seguir, converter os dados fotométricos para o formato HATS. Como o DES DR2 é o nosso catálogo "esquerdo", não precisamos gerar um cache de margem para ele.

Convertendo os dados para o formato hats (**DESCOMENTE A ÚLTIMA LINHA PARA RODAR O PIPELINE**):

In [ ]:
des_args = ImportArguments(
    sort_columns="coadd_object_id",
    ra_column="ra",
    dec_column="dec",
    input_path=DES_DIR,
    file_reader="parquet",
    output_artifact_name=DES_HATS_NAME,
    output_path=HATS_DIR,
    pixel_threshold=30_000,
)

### RUN THE PIPELINE (it will fail if the HATS catalog already exists) 
pipeline_with_client(des_args, client)

Fazendo o plot dos pixels:

In [ ]:
# Read the HATS catalog metadata, it does not load any data, just healpix pixels and other metadata
des_hats_catalog = hats.read_hats(DES_HATS_DIR)
plot_pixels(des_hats_catalog)

# Utilizando o LSDB

"A biblioteca LSDB é um *framework* que facilita e permite a análise espacial rápida de catálogos astronômicos extremamente grandes (ou seja, consulta e cruzamento de O(1B) fontes). Ela visa abordar os desafios do processamento de dados em larga escala, em particular aqueles levantados pelo LSST.

Construída sobre o Dask para escalar e paralelizar operações de forma eficiente em vários *dask workers*, ela aproveita o formato de dados HATS para levantamentos em uma estrutura particionada HEALPix." ([LSDB Docs](https://lsdb.readthedocs.io/en/stable/index.html))

## Instalação do LSDB


A biblioteca LSDB normalmente pode ser instalada via conda ou pip com os comandos:

```shell
conda install -c conda-forge lsdb
```

```shell
python -m pip install lsdb
```

Para mais detalhes sobre a instalação, veja a documentação [neste link](https://lsdb.readthedocs.io/en/stable/installation.html).

## X-matching usando o LSDB

Para os passos a seguir, foi utilizado, principalmente, o seguinte exemplo de referência: <br>
https://lsdb.readthedocs.io/en/stable/tutorials/pre_executed/des-gaia.html <br>

Definimos, a seguir, o nome do diretório que armazenará os dados do crossmatch.

In [ ]:
XMATCH_NAME = "des_dr2_x_2dflens"

OUTPUT_HATS_DIR = HATS_DIR / XMATCH_NAME

A seguir, lemos os dados espectroscópicos e fotométricos, previamente salvos no formato hats.

In [ ]:
des_catalog = lsdb.read_hats(DES_HATS_DIR)

_2dflens_catalog = lsdb.read_hats(_2DFLENS_HATS_DIR, margin_cache=_2DFLENS_MARGIN_CACHE_DIR)

Agora, vamos planejar o crossmatching utilizando o LSDB, mas ainda não iremos executá-lo. Lembrando, novamente, que há uma diferença entre qual é o catálogo "direito" e qual é o "esquerdo" no crossmatching, como dito na seção anterior quando elaborarmos o cache de margem para o 2dFLenS. No nosso caso, o DES DR2 será o nosso catálogo "esquerdo", e o 2dFLenS o nosso catálogo "direito". Assim, usamos o método ```crossmatch``` sobre o catálogo do DES e passamos, como argumento, o catálogo do 2dFLenS. Os demais argumentos são o raio de busca (```radius_arcsec```), em arcsec, o número de objetos vizinhos (```n_neighbors```) que serão encontrados no catálogo da direita para cada objeto no catálogo da esquerda (o padrão é apenas um objeto vizinho, ou seja, o objeto do catálogo direito mais próximo ao objeto em questão no catálogo esquerdo) e os sufixos (```suffixes```) que serão usados para diferenciar os dados de ambos os catálogos nos resultados do crossmatching.

Uma observação importante é que o **raio do crossmatching (radius_arcsec) não pode ser maior que o margin_threshold do cache de margem** do catálogo direito, senão o lsdb exibe um erro:
```bash
ValueError: Cross match radius is greater than margin threshold.
```

In [ ]:
xmatched = des_catalog.crossmatch(
    _2dflens_catalog,
    # Up to 1 arcsec distance, it is the default
    radius_arcsec=2.0,
    # Single closest object, it is the default
    n_neighbors=1,
    # Default would be to use names of the HATS catalogs
    suffixes=("_des", "_2dflens"),
)

display(des_catalog)
display(_2dflens_catalog)
display(xmatched)

Agora, vamos executar o pipeline do crossmatching utilizando o cliente Dask (**DESCOMENTE A LINHA PARA RODAR O PIPELINE**).

In [ ]:
# Run the pipeline with Dask client, it will take a while
xmatched.to_hats(OUTPUT_HATS_DIR, overwrite=True)

A seguir, temos as primeiras linhas da tabela contendo os dados do crossmatching.

In [ ]:
# Look into the data
xmatched_from_disk = lsdb.read_hats(OUTPUT_HATS_DIR)
xmatched_from_disk.head()

Podemos obter também a estatística básica dos dados da tabela.

In [ ]:
df = xmatched_from_disk.compute()

In [ ]:
df.describe()

## Análise dos resultados do X-matching

A seguir, vamos selecionar uma região do céu delimitada por:

In [ ]:
print(f"R.A. min: {ra_min}")
print(f"R.A. max: {ra_max}")
print(f"DEC min: {dec_min}")
print(f"DEC max: {dec_max}")

Para isso, podemos usar o método ```polygon_search``` do LSDB para selecionar essa região de interesse nos catálogos, e depois executar o cálculo com o ```.compute()```.

In [ ]:
polygon_coords = [[ra_min, dec_max], [ra_max, dec_max], [ra_max, dec_min], [ra_min, dec_min]]

des_box = des_catalog.polygon_search(polygon_coords).compute()
_2dflens_box = _2dflens_catalog.polygon_search(polygon_coords).compute()
xmatch_box = xmatched.polygon_search(polygon_coords).compute()

Convertemos, então, as coordenadas R.A. para o intervalo $(-180^{\circ}, 180^{\circ}]$.

In [ ]:
ra_des = np.where(des_box["ra"] > 180, des_box["ra"] - 360, des_box["ra"])
ra_2dflens = np.where(_2dflens_box["ra"] > 180, _2dflens_box["ra"] - 360, _2dflens_box["ra"])
ra_x_2dflens = np.where(xmatch_box["ra_2dflens"] > 180, xmatch_box["ra_2dflens"] - 360, xmatch_box["ra_2dflens"])

Finalmente, montamos o plot. Nesse plot, os dados do DES são representados por pontos azuis, os dados do 2dFLenS por pontos verdes e os objetos do 2dFLenS que obtiveram uma correspondência a algum objeto do DES no crossmatching estão marcados em vermelho.

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(ra_des, des_box["dec"], s=2, alpha=0.01, marker="+", color="blue", label="DES")
plt.scatter(ra_2dflens, _2dflens_box["dec"], s=50, alpha=0.7, color="green", label="2dflens")
plt.scatter(ra_x_2dflens, xmatch_box["dec_2dflens"], s=10, alpha=0.8, color="red", label="x-matched")
plt.xlabel("R.A. (deg)")
plt.ylabel("DEC (deg)")
plt.legend(loc='lower right')

Podemos fazer também um gráfico com o Holoviews exibindo apenas os objetos do DES que obtiveram uma correspondência a algum objeto do 2dFLenS no crossmatching. O interessante desse gráfico é que podemos interagir com ele, utilizando, por exemplo, a ferramenta Hover para obter as coordenadas R.A. e DEC de um determinado objeto e a distância dele ao objeto do 2dFLenS com o qual ele foi associado no crossmatching. Adicionamos, ainda, uma barra de cor que indica a distância dos objetos do DES ao seu correpondente do 2dFLenS, de acordo com o crossmatching.

In [ ]:
# Criar o gráfico 2D
points = hv.Points(df, kdims=['ra_des', 'dec_des'], vdims=['_dist_arcsec'])

# Configurar a coloração e ajustar o tamanho dos labels dos eixos
points.opts(
    width=750, height=500, 
    color='_dist_arcsec', cmap='Viridis', colorbar=False, tools=['hover'], 
    size=8, 
    xlabel='RA (deg)', ylabel='DEC (deg)',
    fontsize={'xticks': 12, 'yticks': 12, 'xlabel': 14, 'ylabel': 14}
)

# Renderizar o gráfico
plot = hv.render(points)

# Personalizar o ColorBar
color_mapper = LinearColorMapper(palette=Viridis256, low=df['_dist_arcsec'].min(), high=df['_dist_arcsec'].max())
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Dist. (Arcsec)')
color_bar.title_text_font_size = '14pt'
color_bar.major_label_text_font_size = '12pt'

# Adicionar o color bar ao layout do plot
plot.add_layout(color_bar, 'right')

# Mostrar o gráfico
show(plot)

Podemos também checar se existem objetos de 2dFLenS que foram associados a mais de um objeto do DES, por causa do cache de margem.

In [ ]:
df_duplicates = df[df.duplicated(subset=['ra_2dflens'])]
df_duplicates

In [ ]:
if not df_duplicates.empty:
    for i in np.arange(0,len(df_duplicates),1):
        id_duplicate = df_duplicates['index_2dflens'].values[i]
        print(f"ID do objeto do 2dFLenS: {id_duplicate}")
        print(f"ID dos objetos do DES associados a esse mesmo objeto do 2dFLenS: {df[df['index_2dflens'] == id_duplicate]['coadd_object_id_des'].tolist()}")
        print('\n')

Além disso, podemos fazer também o histograma das distâncias de separação dos objetos do crossmatching.

In [ ]:
# Convertendo a coluna _dist_arcsec para um array numpy.
df['_dist_arcsec'] = df['_dist_arcsec'].to_numpy()

# Criar um Dataset a partir do DataFrame
dataset = hv.Dataset(df, kdims='_dist_arcsec')

# Aplicar a operação de histograma
hist = hv.operation.histogram(dataset, dimension='_dist_arcsec', normed=False, bins=20)

# Personalizar o histograma
hist.opts(
    xlabel='Distância (arcsec)',
    ylabel='Frequência',
    title='Histograma das distâncias de separação',
    color='blue',
    tools=['hover'],
    width=750,
    height=500
)

# Mostrar o histograma
hist